In [71]:
#%pip install scikit-learn
#%pip install nltk
#%pip install gensim
#%pip install unidecode





# Import packages

In [72]:
import pandas as pd
import re
import os
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
sys.path.append('../')
from utils import remove_similar_rows, find_lines_with_player, remove_similar_rows_per_player, map_emoji_to_description, del_patterns, extract_sentence
import nltk
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords, strip_numeric, strip_punctuation, strip_multiple_whitespaces, strip_short
from unidecode import unidecode




# Load Data

In [73]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

In [74]:
# Filter out the Spanish data and reindex
df_es = df[df["language"] == "es"]

#Reset index
df_es = df_es.reset_index(drop=True)
df_es

,data,player,language,publishedAt
0,"{'content': ""DIRECTO\nMercado de fichajes de f...",Exequiel Palacios,es,2023-01-29T18:25:03Z
1,{'content': 'Con el primer mes del 2023 a poco...,Exequiel Palacios,es,2023-01-30T16:52:46Z
2,{'content': 'Deportes\nGustavo Puerta ya no ju...,Exequiel Palacios,es,2023-01-31T20:41:38Z
3,{'content': 'Dólar blue\nAlberto Fernández\nMa...,Exequiel Palacios,es,2023-02-09T18:32:38Z
4,{'content': 'Dólar blue\nAlberto Fernández\nMa...,Exequiel Palacios,es,2023-02-12T21:13:55Z
...,...,...,...,...
270,"{'content': ""Juventus rescató un empate 1-1 fr...",Exequiel Palacios,es,2023-05-11T19:05:33Z
271,"{'content': ""Juventus rescató un empate 1-1 fr...",Exequiel Palacios,es,2023-05-11T18:26:06Z
272,{'content': 'Este jueves se abren las series d...,Exequiel Palacios,es,2023-05-11T13:44:18Z
273,"{'content': 'Con 10 futbolistas argentinos, la...",Exequiel Palacios,es,2023-05-10T16:03:25Z


# Remove similar rows per player

In [75]:
# Remove the similiar rows (The Function is imported from utils on top)
df_es = remove_similar_rows_per_player(df_es, df_es['player'].unique())

# Lower case

In [76]:
data_lower_es = df_es.copy()

data_lower_es['data'] = data_lower_es['data'].str.lower()
data_lower_es['player'] = data_lower_es['player'].str.lower()
data_lower_es.head()

,data,player,language,publishedAt
0,"{'content': ""directo\nmercado de fichajes de f...",exequiel palacios,es,2023-01-29T18:25:03Z
1,{'content': 'con el primer mes del 2023 a poco...,exequiel palacios,es,2023-01-30T16:52:46Z
2,{'content': 'deportes\ngustavo puerta ya no ju...,exequiel palacios,es,2023-01-31T20:41:38Z
3,{'content': 'dólar blue\nalberto fernández\nma...,exequiel palacios,es,2023-02-09T18:32:38Z
4,{'content': 'dólar blue\nalberto fernández\nma...,exequiel palacios,es,2023-02-12T21:13:55Z


# Romove Patterns

In [77]:
patternlist = [
    'content',
    'directo',
    'espacio publicitario',
    'copyright',
    'foto:',
    'todos los derechos reservados',
    'derechos reservados',
    'suscribete',
    'queda prohibida la reproducción',
    'parcial, por',
    'cualquier medio, de todos los contenidos sin autorización expresa de grupo el comercio',
    'pic.twitter.com'
   
]

In [78]:
# Create a copy
data_wo_pattern = data_lower_es.copy()

#delete patterns
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: del_patterns(str(x), patternlist))

In [79]:
# Replace '\\xaO' with a whitespace in the 'data' column
data_wo_pattern['data'] = data_wo_pattern['data'].str.replace(r'\\xa0', ' ')

# Emojis analysis

In [80]:
"""
# Unicode ranges for emojis
emoji_ranges = [
    (0x1F600, 0x1F64F),  # Emoticons
    (0x1F300, 0x1F5FF),  # Miscellaneous symbols and pictographs
    (0x1F680, 0x1F6FF),  # Transport and map symbols
    (0x2600, 0x26FF),    # Miscellaneous symbols
    (0x2700, 0x27BF),    # Dingbats
    (0xFE00, 0xFE0F),    # Variation Selectors
    (0x1F900, 0x1F9FF),  # Supplemental Symbols and Pictographs
    (0x1F1E6, 0x1F1FF)   # Flags
]

# Function to check if a character is an emoji
def is_emoji(character):
    for start, end in emoji_ranges:
        if ord(character) in range(start, end + 1):
            return True
    return False

def is_emoji_remove(character):
    emoji_ord = ord(character)
    for start, end in emoji_ranges:
        if emoji_ord >= start and emoji_ord <= end:
            return True
    return False

# Assuming your dataframe is named 'df'
articles_with_emoji = 0

# Iterate over the rows of the dataframe
for index, row in df_es.iterrows():
    # Counter for emojis in the current article
    emoji_count = 0

    # Iterate over the columns of the current row
    for column in row.index:
        # Check if the cell contains an emoji
        for char in str(row[column]):
            if is_emoji(char):
                emoji_count += 1

    # If there is at least one emoji in the current article, increment the count of articles with emojis
    if emoji_count > 0:
        articles_with_emoji += 1
        print("Emojis found in article", index + 1, ":", emoji_count)

print("Total number of articles with emojis:", articles_with_emoji)

"""

'\n# Unicode ranges for emojis\nemoji_ranges = [\n    (0x1F600, 0x1F64F),  # Emoticons\n    (0x1F300, 0x1F5FF),  # Miscellaneous symbols and pictographs\n    (0x1F680, 0x1F6FF),  # Transport and map symbols\n    (0x2600, 0x26FF),    # Miscellaneous symbols\n    (0x2700, 0x27BF),    # Dingbats\n    (0xFE00, 0xFE0F),    # Variation Selectors\n    (0x1F900, 0x1F9FF),  # Supplemental Symbols and Pictographs\n    (0x1F1E6, 0x1F1FF)   # Flags\n]\n\n# Function to check if a character is an emoji\ndef is_emoji(character):\n    for start, end in emoji_ranges:\n        if ord(character) in range(start, end + 1):\n            return True\n    return False\n\ndef is_emoji_remove(character):\n    emoji_ord = ord(character)\n    for start, end in emoji_ranges:\n        if emoji_ord >= start and emoji_ord <= end:\n            return True\n    return False\n\n# Assuming your dataframe is named \'df\'\narticles_with_emoji = 0\n\n# Iterate over the rows of the dataframe\nfor index, row in df_es.iterrows

# Match emoji description

In [81]:
data_wo_emojis = data_wo_pattern.copy()

data_wo_emojis['data'] = data_wo_emojis['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group(), language = 'es',), str(x)))


# Remove noise

## Remove numeric

In [82]:
data_rm_es = data_wo_emojis.copy()

data_rm_es['data'] = data_rm_es['data'].apply(strip_numeric)

## Remove links

In [83]:
data_rm_es['data'] = data_rm_es['data'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

## Remove multiple white spaces

In [84]:
data_rm_es['data'] = data_rm_es['data'].apply(strip_multiple_whitespaces)

## Remove marks above names

In [85]:
def remove_accents(text):
    return unidecode(text)

In [86]:
data_rm_es['data'] = data_rm_es['data'].apply(lambda x: remove_accents(str(x)))

## Reset index

In [87]:
data_es_clean1 = data_rm_es.copy()

data_es_clean1.reset_index(drop=True, inplace=True)

# Save data clean 1

In [88]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'es_clean_1.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
data_es_clean1.to_csv(file_path, index=False)

# ------------------------

# Preprocess for data clean 2

# ------------------------

# Remove short words

In [89]:
data_wo_short = data_es_clean1.copy()

data_wo_short['data'] = data_wo_short['data'].apply(strip_short)

# Remove stopwords

In [90]:
#Create copy to new df data_rm_sw as in data removed stopwords
data_es_sw = data_wo_short.copy()

#Load stopwords
#nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')



In [91]:
# Create a list of stop words to remove 
stop_words_to_remove = ['ni', 'no', 'sin']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  spanish_stopwords.remove(word)

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=spanish_stopwords)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_es_sw['data'] = data_es_sw['data'].apply(remove_stopwords_from_text)



# Copy for data clean 3

In [92]:
data_es_clean3 = data_es_sw.copy()

# Remove Punctuation

In [93]:
data_wo_pun = data_es_sw.copy()

data_wo_pun['data'] = data_wo_pun['data'].apply(strip_punctuation)

# Save data clean 2

In [94]:
data_es_clean2 = data_wo_pun.copy()

In [95]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'es_clean_2.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
data_es_clean2.to_csv(file_path, index=False)




# ------------------------

# Data condensed

# ------------------------

# Keep only paragraph

In [96]:
data_es_pp = data_es_clean3.copy()

# select only paragraphs which include playernames 
data_es_pp = find_lines_with_player(data_es_pp, data_es_pp['player'].unique(), n_lines = 1)

# Remove empty rows

In [97]:
data_es_er = data_es_pp.replace('', pd.NA)
data_es_er.dropna(inplace=True)
data_es_er

,data,player,language,publishedAt
0,adeyemi firmo primer gol bundesliga florian w...,exequiel palacios,es,2023-01-29T18:25:03Z
1,ultima semana marzo primera ventana partidos i...,exequiel palacios,es,2023-01-30T16:52:46Z
2,gustavo puerta jugara bayer leverkusen bayer ...,exequiel palacios,es,2023-01-31T20:41:38Z
3,"alberto fernandez presidente afa, claudio \'c...",exequiel palacios,es,2023-02-09T18:32:38Z
4,alberto fernandez tambien participaron campeo...,exequiel palacios,es,2023-02-12T21:13:55Z
...,...,...,...,...
68,"minuto, visitante habia llegado arco romano ge...",piero hincapié,es,2023-05-11T21:13:48Z
69,"bayer leverkusen jugo visita roma, partido ida...",piero hincapié,es,2023-05-11T20:56:21Z
70,"ecuatoriano volvera semifinales torneo uefa, a...",piero hincapié,es,2023-05-11T18:30:25Z
71,seleccion ecuador jugara dos ultimos amistosos...,piero hincapié,es,2023-05-10T23:37:55Z


# Remove player name

In [98]:
data_es_pn = data_es_er.copy()

# for every player remove their names from the texts 
for player in data_es_pn['player'].unique():
    f_l_name = player.split()

    # Extracting the first name
    first_name = str(f_l_name[0])

    # Extracting the last name
    last_name = str(f_l_name[1])

    updated_pattern = r"\b(" + first_name.lower() + r"|" + last_name.lower() + r")\b|"


    # Apply the function to the data column
    data_es_pn.loc[data_es_pn['player'] == player, 'data'] = data_es_pn.loc[data_es_pn['player'] == player, 'data'].apply(lambda x: re.sub(updated_pattern, "", str(x)))

# Remove punctuation

In [99]:

data_es_pn['data'] = data_es_pn['data'].apply(strip_punctuation)
data_es_pn['data'] = data_es_pn['data'].apply(strip_multiple_whitespaces)

# Save data condensed

In [100]:
data_es_condense = data_es_pn.copy()

In [101]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'es_clean_condensed.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
data_es_condense.to_csv(file_path, index=False)